# Implementation of Multi-Behaviour Graph Convolution Network (MBGCN) for Yandex Music Recommendation System

The MBGCN architecture was introduced in the [article](https://fi.ee.tsinghua.edu.cn/~gaochen/papers/SIGIR2020-MBGCN.pdf) by Jin et al. and consists of 3 main architectural features:
- **Multi-Graph Representation**: different relation's types (behaviours) are supported so that user-track interactions are represented as set of bipartite graphs, one for each behaviour (e.g., listen, like, dislike).
- **Behaviour-Specific Propagation**: graph convolutions are performed separately for each of the behaviour-specific graph.
- **Behaviour-Aware Aggregation**: the aggregation is performed after the propagation stage flexibly using appropriate mechanism (e.g., summation, mlp, attention). Such a strategy allows the model to understand, how much important each of the behaviour for the target prediction.

The notebook contains the implementation of the MBGCN model, preprocessing the Yambda Dataset, training and evaluation of the model, and visualization of the results.

The main goal is to **create working model training pipeline**. It implies that the high performance, strict objectivity of the training and evaluation, and high quality of the predictions were omitted from the consideration. Based on this, the following actions were taken:

- **Set the "Listen+" setup**: the only listen type of relations with played track ratio not less than 50% was considered. This implies the loss of the advantage of the MBGCN architecture: use several behaviours for more accurate predictions.
- **Sample data**: the sampling of the users and track was performed for speeding up the training.
- **Shrink the number of layers and embedding dimensions**: this was done for speeding up the training.
- **No hyperparameter tuning was performed**: the decision was taken according to the main goal of the notebook.

All these actions leed to the disadvantages listed above. The new experiments will be conducted to close these shortcomings.

---

**Install and import necessary libraries**

In [ ]:
!pip install torch_geometric
!pip install fastparquet

In [2]:
from typing import List, Optional, Dict, Literal, Tuple
from datasets import load_dataset, DatasetDict, Dataset
from torch_geometric.nn import SAGEConv
from torch_geometric.data import Data
from collections import defaultdict
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
from tqdm.auto import tqdm
from torch import Tensor
import torch.nn as nn
import pandas as pd
import numpy as np
import torch

## Data Preprocessing

The Yambda-50m dataset will be used to train and evaluate the model. This dataset contains users, items, and interactions between them. The key feature of the dataset is the presence of "is_organic" field showing that the action was intentional. This dataset also contains relationships of different types: listens, likes, dislikes, unlikes, etc. Also, the semantic embeddings for each track is available.

In the notebook, only the information about the relationship "listen" will be used due to lack of the time and computational resources.

---

**Implementation of the convenient helper class to manage the dataset**

In [3]:
class YambdaDataset:
  INTERACTIONS = frozenset(["likes", "listens", "multi_event", "dislikes", "unlikes", "undislikes"])

  def __init__(
      self,
      dataset_type: Literal["flat", "sequential"] = "flat",
      dataset_size: Literal["50m", "500m", "5b"] = "50m"
  ):
    assert dataset_type in {"flat", "sequential"}
    assert dataset_size in {"50m", "500m", "5b"}
    self.dataset_type = dataset_type
    self.dataset_size = dataset_size

  def interaction(self, event_type: Literal["likes", "listens", "multi_event", "dislikes", "unlikes", "undislikes"]) -> Dataset:
    assert event_type in self.INTERACTIONS
    return self._download(f"{self.dataset_type}/{self.dataset_size}", event_type)

  def audio_embeddings(self) -> Dataset:
    return self._download("", "embeddings")

  def album_item_mapping(self) -> Dataset:
    return self._download("", "album_item_mapping")

  def artist_item_mapping(self) -> Dataset:
    return self._download("", "artist_item_mapping")

  @staticmethod
  def _download(data_dir: str, file: str) -> Dataset:
    data = load_dataset("yandex/yambda", data_dir=data_dir, data_files=f"{file}.parquet")
    assert isinstance(data, DatasetDict)
    return data["train"]

---

**Implementation of the preprocessing function**

In [4]:
def preprocess_data(
    data_path: str,
    behaviours: List[str] = None,
    num_sampled_users: int = 3000,
    max_sampled_items: int = 50000,
    item_feature_dim: int = 128,
) -> Tuple[Data, Dict, Dict, Dict]:
    if behaviours is None:
        behaviours = ["listens"]

    print("Data downloading...")
    dataset = YambdaDataset("flat", "50m")

    all_interactions = []
    for behaviour in behaviours:
        df = dataset.interaction(behaviour).to_pandas()
        if behaviour == 'listens':
            df = df[df['played_ratio_pct'] >= 50]
        df['behaviour'] = behaviour
        all_interactions.append(df[['uid', 'item_id', 'behaviour']])

    interactions_df = pd.concat(all_interactions, ignore_index=True)
    print(f"Data downloaded. Number of interactions: {len(interactions_df)}")


    print(f"Sampling...")
    all_item_ids = interactions_df['item_id'].unique()
    sampled_item_ids = np.random.choice(all_item_ids, size=min(max_sampled_items, len(all_item_ids)), replace=False)

    sampled_df = interactions_df[interactions_df['item_id'].isin(sampled_item_ids)]

    all_user_ids = sampled_df['uid'].unique()
    sampled_user_ids = np.random.choice(all_user_ids, size=min(num_sampled_users, len(all_user_ids)), replace=False)
    sampled_df = sampled_df[sampled_df['uid'].isin(sampled_user_ids)]

    final_user_ids = sampled_df['uid'].unique()
    final_item_ids = sampled_df['item_id'].unique()
    print(f"Sampling done. Number of users: {len(final_user_ids)}. Number of tracks: {len(final_item_ids)}. Number of interactions: {len(sampled_df)}")


    # Create mapping between objects and their index
    user_to_idx = {uid: i for i, uid in enumerate(final_user_ids)}
    item_to_idx = {iid: i for i, iid in enumerate(final_item_ids)}
    behaviours_map = {name: i for i, name in enumerate(behaviours)}

    num_users = len(user_to_idx)
    num_items = len(item_to_idx)


    # Create random embeddings for items
    item_feats = torch.randn(num_items, item_feature_dim)

    sampled_df['u_idx'] = sampled_df['uid'].map(user_to_idx)
    sampled_df['i_idx'] = sampled_df['item_id'].map(item_to_idx)
    sampled_df['b_idx'] = sampled_df['behaviour'].map(behaviours_map)

    u = torch.tensor(sampled_df['u_idx'].values, dtype=torch.long)
    v = torch.tensor(sampled_df['i_idx'].values, dtype=torch.long) + num_users
    b = torch.tensor(sampled_df['b_idx'].values, dtype=torch.long)

    src = torch.cat([u, v], dim=0)
    dst = torch.cat([v, u], dim=0)
    edge_index = torch.stack([src, dst], dim=0)
    edge_type = torch.cat([b, b], dim=0)


    graph_data = Data(
        num_users=num_users,
        num_items=num_items,
        item_feats=item_feats,
        edge_index=edge_index,
        edge_type=edge_type
    )

    print(f"Dataset is preprocessed.")
    return graph_data, user_to_idx, item_to_idx, behaviours_map

## MBGCN Model Implementation

---

**GNN Block for processing one behaviour-specific bipartite graph**

In [5]:
class BehaviourGNNBlock(nn.Module):

    def __init__(self, in_dim: int, hidden_dim: int, n_layers: int, dropout: float = 0.2):
        super().__init__()
        self.n_layers = n_layers
        self.convs = nn.ModuleList()
        for i in range(n_layers):
            in_d = in_dim if i == 0 else hidden_dim
            self.convs.append(SAGEConv(in_d, hidden_dim))

        self.bn = nn.ModuleList([nn.BatchNorm1d(hidden_dim) for _ in range(n_layers)])
        self.dropout = dropout

    def forward(self, x: Tensor, edge_index: Tensor, edge_weight: Optional[Tensor] = None):
        h = x
        for i, conv in enumerate(self.convs):
            h = conv(h, edge_index)
            h = self.bn[i](h)

            if i < self.n_layers - 1:
                h = F.relu(h)
                h = F.dropout(h, p=self.dropout, training=self.training)

        return h

---

**Main implementation of the MBGCN architecture**

In [6]:
class MBGCN(nn.Module):

    def __init__(
        self,
        num_users: int,
        num_items: int,
        item_feat_dim: int,
        node_emb_dim: int = 128,
        user_init_dim: int = 64,
        n_gnn_layers: int = 2,
        behaviours: Optional[List[str]] = None,
        behaviour_hidden_dim: int = 128,
        fusion: str = "attention",
        fusion_heads: int = 4,
        dropout: float = 0.2,
        use_batchnorm: bool = True,
        score_mlp_hidden: int = 128,
    ):
        super().__init__()
        assert behaviours is not None and len(behaviours) > 0, "Behaviours list must be non-empty"
        self.num_users = num_users
        self.num_items = num_items
        self.num_nodes = num_users + num_items
        self.behaviours = behaviours
        self.B = len(behaviours)
        self.node_emb_dim = node_emb_dim

        self.user_emb = nn.Embedding(num_users, user_init_dim)
        self.user_proj = nn.Linear(user_init_dim, node_emb_dim)
        self.item_proj = nn.Linear(item_feat_dim, node_emb_dim)

        # behaviour-specific GNN blocks
        self.behavior_blocks = nn.ModuleList([
            BehaviourGNNBlock(in_dim=node_emb_dim, hidden_dim=behaviour_hidden_dim, n_layers=n_gnn_layers, dropout=dropout)
            for _ in range(self.B)
        ])

        # project behaviour hidden -> node_emb_dim
        if behaviour_hidden_dim != node_emb_dim:
            self.behav_to_node = nn.ModuleList([nn.Linear(behaviour_hidden_dim, node_emb_dim) for _ in range(self.B)])
        else:
            self.behav_to_node = None

        # fuse the outputs of all the behaviour blocks
        allowed_fusions = {"sum", "concat", "attention", "gating"}
        assert fusion in allowed_fusions, f"fusion must be one of {allowed_fusions}"
        self.fusion = fusion
        if fusion == "attention":
            self.query_proj = nn.Linear(node_emb_dim, node_emb_dim)
            self.key_projs = nn.ModuleList([nn.Linear(node_emb_dim, node_emb_dim) for _ in range(self.B)])
            self.fuse_linear = nn.Linear(node_emb_dim, node_emb_dim)
        elif fusion == "gating":
            self.gate_linears = nn.ModuleList([nn.Linear(2 * node_emb_dim, node_emb_dim) for _ in range(self.B)])
            self.fuse_linear = nn.Linear(node_emb_dim, node_emb_dim)
        elif fusion == "concat":
            self.fuse_linear = nn.Linear(self.B * node_emb_dim, node_emb_dim)
        else:
            self.fuse_linear = nn.Identity()

        # Final refinement
        self.refine = nn.Sequential(
            nn.Linear(node_emb_dim, node_emb_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
        )

        # Scoring head
        self.score_mlp = nn.Sequential(
            nn.Linear(2 * node_emb_dim, score_mlp_hidden),
            nn.ReLU(),
            nn.Linear(score_mlp_hidden, 1),
        )

        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.user_emb.weight)
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Embedding):
                 nn.init.xavier_uniform_(m.weight)


    def _build_initial_node_features(self, item_feats: Tensor) -> Tensor:
        device = item_feats.device
        users = self.user_emb.weight
        users = self.user_proj(users)
        items = self.item_proj(item_feats)
        x = torch.cat([users, items], dim=0)
        return x.to(device)


    def forward(self, item_feats: Tensor, edge_index: Tensor, edge_type: Tensor, edge_attr: Optional[Dict[str, Tensor]] = None) -> Tensor:
        x0 = self._build_initial_node_features(item_feats)

        behav_outputs = []
        for b in range(self.B):
            mask = (edge_type == b)
            if mask.sum() == 0:
                behav_outputs.append(x0)
                continue

            edge_index_b = edge_index[:, mask]

            block = self.behavior_blocks[b]
            behav_h = block(x0, edge_index_b)

            if self.behav_to_node is not None:
                behav_h = self.behav_to_node[b](behav_h)

            behav_h = x0 + behav_h
            behav_outputs.append(behav_h)

        stack = torch.stack(behav_outputs, dim=0)

        if self.fusion == 'sum':
            fused = torch.sum(stack, dim=0)
            fused = self.fuse_linear(fused)
        elif self.fusion == 'concat':
            concat = stack.permute(1, 0, 2).contiguous().view(self.num_nodes, -1)
            fused = self.fuse_linear(concat)
        elif self.fusion == 'gating':
            fused = 0
            for b in range(self.B):
                beh = stack[b]
                gate_input = torch.cat([x0, beh], dim=-1)
                gate = torch.sigmoid(self.gate_linears[b](gate_input))
                gated = gate * beh
                if isinstance(fused, int):
                    fused = gated
                else:
                    fused = fused + gated
            fused = self.fuse_linear(fused)
        elif self.fusion == 'attention':
            Q = self.query_proj(x0)
            attn_logits = []
            for b in range(self.B):
                Kb = self.key_projs[b](stack[b])
                log = (Q * Kb).sum(dim=-1, keepdim=True)
                attn_logits.append(log)

            attn_logits = torch.cat(attn_logits, dim=-1)
            attn_weights = F.softmax(attn_logits, dim=-1)

            attn_weights = attn_weights.unsqueeze(-1)
            stack_perm = stack.permute(1, 0, 2)
            fused = (attn_weights * stack_perm).sum(dim=1)
            fused = self.fuse_linear(fused)
        else:
            raise RuntimeError('Unknown fusion')

        final_emb = self.refine(fused)
        return final_emb

    def score(self, user_idx: Tensor, item_idx: Tensor, node_emb: Tensor, mode: str = 'mlp') -> Tensor:
        u = node_emb[user_idx]
        i = node_emb[item_idx]
        if mode == 'dot':
            return (u * i).sum(dim=-1)
        elif mode == 'mlp':
            x = torch.cat([u, i], dim=-1)
            return self.score_mlp(x).squeeze(-1)
        else:
            raise ValueError('mode must be dot or mlp')

    def predict_batch(self, node_emb: Tensor, users: Tensor, candidates: Tensor, batch_size: int = 4096, mode: str = 'mlp') -> Tensor:
        device = node_emb.device
        scores = []
        n = users.size(0)
        for i in range(0, n, batch_size):
            batch_u = users[i:i+batch_size].to(device)
            batch_c = candidates[i:i+batch_size].to(device)
            scores.append(self.score(batch_u, batch_c, node_emb, mode=mode).detach().cpu())
        return torch.cat(scores, dim=0)

## Training and Evaluation the model

---

**Split the data to train/val/test**

The main disadvantage of the current split function is that it is not time-aware. It means that the model can know the future actions of the user from the training to predict current interaction. The authors of the Yambda dataset suggested another global time-aware splitting, but due to sampling is was not implemented here.

In [7]:
def split_interactions(data: Data, val_ratio: float = 0.1, test_ratio: float = 0.1) -> tuple:
    num_interactions = data.edge_index.size(1)
    indices = np.arange(num_interactions)
    np.random.shuffle(indices)

    val_size = int(num_interactions * val_ratio)
    test_size = int(num_interactions * test_ratio)
    train_size = num_interactions - val_size - test_size

    train_indices = torch.from_numpy(indices[:train_size]).long()
    val_indices = torch.from_numpy(indices[train_size: train_size + val_size]).long()
    test_indices = torch.from_numpy(indices[train_size + val_size:]).long()

    train_user_items = {}
    train_edge_index = data.edge_index[:, train_indices]

    user_nodes = train_edge_index[0].tolist()
    dst_nodes = train_edge_index[1].tolist()

    for u, dst in zip(user_nodes, dst_nodes):
        if int(dst) >= int(data.num_users):
            item_rel = int(dst) - int(data.num_users)
            if int(u) not in train_user_items:
                train_user_items[int(u)] = set()
            train_user_items[int(u)].add(item_rel)
        else:
            continue

    print(f"Split done. Train length: {len(train_indices)}, val length: {len(val_indices)}, test length: {len(test_indices)}")
    return train_indices, val_indices, test_indices, train_user_items


---

**Sample negative examples for the training (BPR loss is used)**

In [8]:
def sample_negatives(users, train_user_pos, num_items, device):
    batch_size = users.size(0)
    negs = torch.empty(batch_size, dtype=torch.long, device=device)
    remaining = torch.ones(batch_size, dtype=torch.bool, device=device)

    while remaining.any():
        cand = torch.randint(0, num_items, (batch_size,), device=device)
        for i in torch.nonzero(remaining, as_tuple=False).view(-1).tolist():
            u = int(users[i].item())
            rel = int(cand[i].item())
            if rel in train_user_pos.get(u, ()):
                pass
            else:
                negs[i] = rel
                remaining[i] = False
    return negs

---

**Construct mapping between users and the interacted items**

In [9]:
def build_user_pos_dict(data, edge_pos_indices):
    user_pos = defaultdict(set)
    ei = data.edge_index
    for pos in edge_pos_indices:
        u = int(ei[0, pos].item())
        item_abs = int(ei[1, pos].item())
        item_rel = item_abs - data.num_users
        user_pos[u].add(item_rel)
    return user_pos

---

**Implementation of the train function**

In [10]:
def train(model: torch.nn.Module,
          data: Data,
          train_edge_indices: torch.LongTensor,
          optimizer: torch.optim.Optimizer,
          train_user_pos: dict,
          batch_size: int = 1024,
          device: torch.device = None):
    if device is None:
        device = next(model.parameters()).device

    model.train()
    total_loss = 0.0
    num_batches = 0

    if train_edge_indices.device != torch.device('cpu'):
        train_edge_indices_cpu = train_edge_indices.cpu()
    else:
        train_edge_indices_cpu = train_edge_indices

    perm = train_edge_indices_cpu[torch.randperm(len(train_edge_indices_cpu))]

    train_edge_index_full = data.edge_index[:, train_edge_indices].to(device)
    train_edge_type_full  = data.edge_type[train_edge_indices].to(device)

    from tqdm.auto import tqdm
    it = range(0, len(perm), batch_size)
    for start in tqdm(it, desc="train batches"):
        batch_edges_cpu = perm[start:start+batch_size]
        batch_edges = batch_edges_cpu.to(data.edge_index.device)

        node_emb = model(data.item_feats.to(device), train_edge_index_full, train_edge_type_full)

        users = data.edge_index[0, batch_edges].long().to(device)
        pos_items_abs = data.edge_index[1, batch_edges].long().to(device)

        neg_items_rel = sample_negatives(users, train_user_pos, data.num_items, device)
        neg_items_abs = neg_items_rel + data.num_users

        user_emb = node_emb[users]
        pos_emb  = node_emb[pos_items_abs]
        neg_emb  = node_emb[neg_items_abs]

        pos_scores = (user_emb * pos_emb).sum(dim=-1)
        neg_scores = (user_emb * neg_emb).sum(dim=-1)

        loss = -torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-10).mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += float(loss.item())
        num_batches += 1

    avg_loss = total_loss / max(1, num_batches)
    return avg_loss

---

**Implementation of the test function**

In [11]:
@torch.no_grad()
def test(model: torch.nn.Module,
         data: Data,
         train_edge_indices: torch.LongTensor,
         eval_edge_indices: torch.LongTensor,
         train_user_pos: dict,
         k: int = 20,
         device: torch.device = None,
         batch_user_size: int = 128):
    if device is None:
        device = next(model.parameters()).device

    model.eval()

    train_ei = data.edge_index[:, train_edge_indices].to(device)
    train_et = data.edge_type[train_edge_indices].to(device)

    node_emb = model(data.item_feats.to(device), train_ei, train_et)
    user_emb = node_emb[:data.num_users]
    item_emb = node_emb[data.num_users:]

    eval_edges = data.edge_index[:, eval_edge_indices]
    users_eval = eval_edges[0].cpu().numpy().tolist()
    dst_eval = eval_edges[1].cpu().numpy().tolist()

    gt = {}
    for u, dst in zip(users_eval, dst_eval):
        if int(dst) >= int(data.num_users):
            item_rel = int(dst) - int(data.num_users)
            gt.setdefault(int(u), set()).add(item_rel)

    users_list = sorted(gt.keys())
    all_recalls, all_ndcgs = [], []

    from tqdm.auto import tqdm
    for i in tqdm(range(0, len(users_list), batch_user_size), desc="test batches"):
        batch_users = users_list[i:i+batch_user_size]
        ub = torch.tensor(batch_users, dtype=torch.long, device=device)
        u_emb = user_emb[ub]

        scores = torch.matmul(u_emb, item_emb.t())

        mask_rows = []
        mask_indices = []
        for row_idx, uid in enumerate(batch_users):
            pos_set = train_user_pos.get(int(uid), ())
            if not pos_set:
                continue
            filtered = [int(x) for x in pos_set if 0 <= int(x) < int(data.num_items)]
            if not filtered:
                continue
            mask_rows.extend([row_idx] * len(filtered))
            mask_indices.extend(filtered)

        if len(mask_rows) > 0:
            mask_rows = torch.tensor(mask_rows, dtype=torch.long)
            mask_indices = torch.tensor(mask_indices, dtype=torch.long)
            mask_rows = mask_rows.to(device)
            mask_indices = mask_indices.to(device)
            if torch.any(mask_indices < 0) or torch.any(mask_indices >= data.num_items):
                raise RuntimeError("Mask indices out of range: check train_user_pos or split logic.")
            scores[mask_rows, mask_indices] = -1e9

        topk_vals, topk_inds = torch.topk(scores, k, dim=1)
        topk_inds = topk_inds.cpu().numpy()

        for j, uid in enumerate(batch_users):
            recs = set([int(x) for x in topk_inds[j]])
            relevant = gt.get(int(uid), set())
            hits = len(recs & relevant)
            recall = hits / len(relevant) if len(relevant) > 0 else 0.0
            all_recalls.append(recall)

            dcg = 0.0
            for rank, item in enumerate(topk_inds[j]):
                if int(item) in relevant:
                    dcg += 1.0 / np.log2(rank + 2.0)
            idcg = sum([1.0 / np.log2(x + 2.0) for x in range(min(len(relevant), k))]) if len(relevant) > 0 else 0.0
            ndcg = (dcg / idcg) if idcg > 0 else 0.0
            all_ndcgs.append(ndcg)

    return float(np.mean(all_recalls)) if all_recalls else 0.0, float(np.mean(all_ndcgs)) if all_ndcgs else 0.0


---

**The main training loop**

In [14]:
DATA_ROOT_PATH = '../input/yambda/flat/50m/'
target_behaviours = ["likes", "listens", "dislikes"]

graph, user_map, item_map, behaviour_map = preprocess_data(
    data_path=DATA_ROOT_PATH,
    behaviours=target_behaviours,
    num_sampled_users=1000,
    max_sampled_items=80000,
    item_feature_dim=128
)

train_idx, val_idx, test_idx, train_user_pos = split_interactions(graph, val_ratio=0.1, test_ratio=0.1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 5
lr = 1e-3
batch_size = 2048
k_eval = 20
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

model = MBGCN(
    num_users=graph.num_users,
    num_items=graph.num_items,
    item_feat_dim=128,
    node_emb_dim=128,
    user_init_dim=64,
    n_gnn_layers=2,
    behaviours=target_behaviours,
    behaviour_hidden_dim=128,
    fusion="attention",
    dropout=0.2
)

train_edge_indices = torch.tensor(train_idx, dtype=torch.long) if not isinstance(train_idx, torch.Tensor) else train_idx.long()
val_edge_indices   = torch.tensor(val_idx, dtype=torch.long)   if not isinstance(val_idx, torch.Tensor) else val_idx.long()
test_edge_indices  = torch.tensor(test_idx, dtype=torch.long)  if not isinstance(test_idx, torch.Tensor) else test_idx.long()

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

train_user_pos = build_user_pos_dict(graph, train_edge_indices)

history = {
    'loss': [],
    'val_recall': [],
    'val_ndcg': []
}

best_val_ndcg = -1.0
best_epoch = -1

for epoch in range(1, epochs + 1):
    avg_loss = train(
        model=model,
        data=graph,
        train_edge_indices=train_edge_indices,
        optimizer=optimizer,
        train_user_pos=train_user_pos,
        batch_size=1024,
        device=device
    )

    val_recall, val_ndcg = test(
        model=model,
        data=graph,
        train_edge_indices=train_edge_indices,
        eval_edge_indices=val_edge_indices,
        train_user_pos=train_user_pos,
        k=k_eval,
        device=device,
        batch_user_size=128
    )

    history['loss'].append(avg_loss)
    history['val_recall'].append(val_recall)
    history['val_ndcg'].append(val_ndcg)

    print(f"Epoch {epoch:03d} loss={avg_loss:.6f} val_rec@{k_eval}={val_recall:.4f} val_ndcg@{k_eval}={val_ndcg:.4f}")

Data downloading...


Generating train split: 0 examples [00:00, ? examples/s]

flat/50m/dislikes.parquet:   0%|          | 0.00/990k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Data downloaded. Number of interactions: 30428510
Sampling...
Sampling done. Number of users: 1000. Number of tracks: 24083. Number of interactions: 370788
Dataset is preprocessed.
Split done. Train length: 593262, val length: 74157, test length: 74157


...

test batches:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 004 loss=0.215821 val_rec@20=0.0236 val_ndcg@20=0.0215


train batches:   0%|          | 0/580 [00:00<?, ?it/s]

test batches:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 005 loss=0.208350 val_rec@20=0.0260 val_ndcg@20=0.0222


## Results Visualization

In [15]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set_style("whitegrid")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

epochs_range = range(1, epochs + 1)
ax1.plot(epochs_range, history['loss'], 'o-', label='Training Loss', color='b')
ax1.set_title('Training Loss Dynamics (Loss)', fontsize=16)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.legend(fontsize=12)
ax1.tick_params(axis='both', which='major', labelsize=10)
ax1.xaxis.set_major_locator(plt.MaxNLocator(integer=True))

ax2.plot(epochs_range, history['val_recall'], 's-', label=f'Validation Recall@{k_eval}', color='g')
ax2.plot(epochs_range, history['val_ndcg'], 'D-', label=f'Validation NDCG@{k_eval}', color='r')
ax2.set_title('Metrics on validation set', fontsize=16)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Value', fontsize=12)
ax2.legend(fontsize=12)
ax2.tick_params(axis='both', which='major', labelsize=10)
ax2.xaxis.set_major_locator(plt.MaxNLocator(integer=True))

plt.tight_layout()
plt.show()